In [1]:
#Comparing difference MA strategies in a 5 year period
import pandas as pd
import yfinance as yf
from datetime import datetime, timedelta

def compare_ma_strategies():
    end_date = '2025-08-08'
    end_dt = datetime.strptime(end_date, '%Y-%m-%d')
    start_date = '2020-08-08'
    initial_capital = 1000
    
    ma_pairs = [
        ('MA10_MA50', 10, 50),
        ('MA20_MA100', 20, 100),
        ('MA50_MA150', 50, 150),
        ('MA100_MA200', 100, 200),
        ('MA50_MA200', 50, 200) 
    ]
    
    data = yf.download("^GSPC", start = start_date, end = end_date, auto_adjust = True)
    data = data[['Close']].dropna()
    
    results = []

    for pair_name, short_ma, long_ma in ma_pairs:
        print(f"\nTesting {pair_name} strategy...")
        
        df = data.copy()
        df[f'ma{short_ma}'] = df['Close'].rolling(short_ma).mean()
        df[f'ma{long_ma}'] = df['Close'].rolling(long_ma).mean()
        df.dropna(inplace=True)

        df['signal'] = 0
        df.loc[(df[f'ma{short_ma}'].shift(1) < df[f'ma{long_ma}'].shift(1)) & 
               (df[f'ma{short_ma}'] > df[f'ma{long_ma}']), 'signal'] = 2 
        df.loc[(df[f'ma{short_ma}'].shift(1) > df[f'ma{long_ma}'].shift(1)) & 
               (df[f'ma{short_ma}'] < df[f'ma{long_ma}']), 'signal'] = 1 

        position = 0  
        entry_price = 0.0
        cash = initial_capital
        portfolio_value = []
        trades = []
        
        for i in range(len(df)):
            current_price = df['Close'].iloc[i].item()
            current_signal = int(df['signal'].iloc[i])
            
            if position == 1 and current_signal == 2: 
                pnl = initial_capital * (entry_price / current_price - 1)
                cash += pnl
                trades.append(('Close Short', df.index[i], current_price, pnl))
                position = 2
                entry_price = current_price
                trades.append(('Open Long', df.index[i], current_price, 0))
            elif position == 2 and current_signal == 1: 
                pnl = initial_capital * (current_price / entry_price - 1)
                cash += pnl
                trades.append(('Close Long', df.index[i], current_price, pnl))
                position = 1
                entry_price = current_price
                trades.append(('Open Short', df.index[i], current_price, 0))
            elif position == 0: 
                if current_signal == 1:
                    position = 1
                    entry_price = current_price
                    cash -= initial_capital
                    trades.append(('Open Short', df.index[i], current_price, 0))
                elif current_signal == 2:
                    position = 2
                    entry_price = current_price
                    cash -= initial_capital
                    trades.append(('Open Long', df.index[i], current_price, 0))
            
            if position == 1: 
                position_value = initial_capital * (entry_price / current_price)
            elif position == 2: 
                position_value = initial_capital * (current_price / entry_price)
            else:
                position_value = 0.0
            
            portfolio_value.append(cash + position_value)
        
        final_value = portfolio_value[-1]
        total_return_pct = (final_value - initial_capital) / initial_capital * 100
        num_trades = len([t for t in trades if t[0].startswith('Close')])
        annualized_return = ((final_value / initial_capital) ** (1/20) - 1) * 100
        
        peak = initial_capital
        max_drawdown = 0
        for value in portfolio_value:
            if value > peak:
                peak = value
            drawdown = (peak - value) / peak * 100
            if drawdown > max_drawdown:
                max_drawdown = drawdown
        
        results.append({
            'MA Pair': pair_name,
            'Short MA': short_ma,
            'Long MA': long_ma,
            'Final Value ($)': round(final_value, 2),
            'Total Return (%)': round(total_return_pct, 2),
            'Annualized Return (%)': round(annualized_return, 2),
            'Number of Trades': num_trades,
            'Max Drawdown (%)': round(max_drawdown, 2),
            'Trades per Year': round(num_trades / 5, 2)
        })
    
    results_df = pd.DataFrame(results).sort_values('Final Value ($)', ascending=False)
    
    results_df.to_csv('ma_strategy_comparison.csv', index=False)
    
    return results_df

performance_comparison = compare_ma_strategies()
print("\nMoving Average Strategy Performance Comparison:")
print(performance_comparison)


[*********************100%***********************]  1 of 1 completed



Testing MA10_MA50 strategy...

Testing MA20_MA100 strategy...

Testing MA50_MA150 strategy...

Testing MA100_MA200 strategy...

Testing MA50_MA200 strategy...

Moving Average Strategy Performance Comparison:
       MA Pair  Short MA  Long MA  Final Value ($)  Total Return (%)  \
3  MA100_MA200       100      200          1466.92             46.69   
0    MA10_MA50        10       50          1362.23             36.22   
4   MA50_MA200        50      200          1186.90             18.69   
2   MA50_MA150        50      150          1130.28             13.03   
1   MA20_MA100        20      100           935.33             -6.47   

   Annualized Return (%)  Number of Trades  Max Drawdown (%)  Trades per Year  
3                   1.93                 2             18.27              0.4  
0                   1.56                27             17.07              5.4  
4                   0.86                 3             21.06              0.6  
2                   0.61              